# Mounting Google Drive 

In [ ]:
# Authorization User

# installing necessary libraries to perform authorization

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
# Mount google drive as disk

!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
# Check contents of folder working with
import os

os.chdir("/content/drive/Emotion_Analysis")
!ls

data  drive  embeddings  models


In [ ]:
# Append new folder/ path of folder to use to perform operations

import sys
sys.path.append('/content/drive/Emotion_Analysis')

In [ ]:
# not working   # Check the content of the new dir/ or folder working on 
# not working   # os.chdir("/content/drive/Emotion_Analysis")
# not working   # os.chdir("/")
# not working   # !ls 

# print out appended working folder path
print(sys.path[-1])

/content/drive/Emotion_Analysis


---

In [ ]:
## =================== 1 =================== 

# 1. Import Data

import nltk
nltk.download('punkt')

import pandas as pd
import numpy as np

# text preprocessing
from nltk.tokenize import word_tokenize
import re

# plots and metrics
import matplotlib.pyplot as plt
##### from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# preparing input to our model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# keras layers
##### from keras.models import Sequential
##### from keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
## =================== 2 =================== 

# Defining vector space dimension and fixed input size
# Number of labels: joy, anger, fear, sadness, neutral
##### num_classes = 5
# Number of dimensions for word embedding
##### embed_num_dims = 300
# Max input length (max number of words) 



max_seq_len = 500
class_names = ['joy', 'fear', 'anger', 'sadness', 'neutral']

In [ ]:
## =================== 3 =================== 

import io

#Append your path
# import sys
# sys.path.append('/content/drive/Emotion_Analysis')


# reference code
# data_train = pd.read_csv('data/data_train.csv', encoding='utf-8')
# data_test = pd.read_csv('data/data_test.csv', encoding='utf-8')
# loaded_csv_file = pd.read_csv(io.BytesIO(data_to_load['filename.csv']))

# data_train = pd.read_csv(io.BytesIO(data_to_load['data_train.csv']), encoding='utf-8')
# data_test = pd.read_csv(io.BytesIO(data_to_load['data_test.csv']), encoding='utf-8')


data_train = pd.read_csv('data/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('data/data_test.csv', encoding='utf-8')


X_train = data_train.Text
X_test = data_test.Text

y_train = data_train.Emotion
y_test = data_test.Emotion

data = data_train.append(data_test, ignore_index=True)

In [ ]:
## =================== 4 =================== 

# defining a function to clean data

def clean_text(data):    
    # remove hashtags and @usernames
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)
    
    # tekenization using nltk
    data = word_tokenize(data)
    
    return data

In [ ]:
## =================== 5 =================== 

texts = [' '.join(clean_text(text)) for text in data.Text]

texts_train = [' '.join(clean_text(text)) for text in X_train]
texts_test = [' '.join(clean_text(text)) for text in X_test]

In [ ]:
## =================== 6 =================== 

# Tokenizing and fitting the model using keras library

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

# vacab size is number of unique words + reserved 0 index for padding
vocab_size = len(index_of_words) + 1

# print('Number of unique words: {}'.format(len(index_of_words)))

In [ ]:
## =================== 7 =================== 
# Padding inputs so each of them have the same length

# We defined maximun number of words for our texts and input size to our model has to be fixed - 
# padding with zeros to keep the same input lenght (longest input in our dataset is ~250 words)

X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len )

X_train_pad

array([[    0,     0,     0, ...,   119,    51,   345],
       [    0,     0,     0, ...,    37,   277,   154],
       [    0,     0,     0, ...,    16,     2,  1210],
       ...,
       [    0,     0,     0, ...,   876,     4,   909],
       [    0,     0,     0, ...,     1,     6,   117],
       [    0,     0,     0, ..., 10259,   173,    13]], dtype=int32)

In [ ]:
## =================== 8 =================== 
# Categorizing labels

encoding = {
    'joy': 0,
    'fear': 1,
    'anger': 2,
    'sadness': 3,
    'neutral': 4
}

# Integer labels
y_train = [encoding[x] for x in data_train.Emotion]
y_test = [encoding[x] for x in data_test.Emotion]

In [ ]:
## =================== 9 =================== 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
## =================== 10 =================== 
# v2 load

from keras.models import load_model
loaded_model_v2 = load_model('/content/drive/Emotion_Analysis/models/bi_gru_w2vec_v2_30eps.h5')

In [ ]:
## =================== 11 =================== 

# time is imported to see how long each prediction took
import time

message1 = ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled"]
message2 = ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled, but now I'm sad that my dad died"]
message3 = ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled, but now I'm sad that my dad died and now I just won the lottery so I will celebrate my victory and I am very thrilled about it"]
message4 = ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled, but now I'm sad that my dad died and now I just won the lottery so I will celebrate my victory and I am very thrilled about it but then found out It all goes to my ex wife. I think I'm depressed now"]
message5 = ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled, but now I'm sad that my dad died and now I just won the lottery so I will celebrate my victory and I am very thrilled about it but then found out It all goes to my ex wife. I am super annoyed right now"]

message6 = ["I'm verry"]

seq1 = tokenizer.texts_to_sequences(message1)
seq2 = tokenizer.texts_to_sequences(message2)
seq3 = tokenizer.texts_to_sequences(message3)
seq4 = tokenizer.texts_to_sequences(message4)
seq5 = tokenizer.texts_to_sequences(message5)
seq6 = tokenizer.texts_to_sequences(message6)

padded1 = pad_sequences(seq1, maxlen=max_seq_len)
padded2 = pad_sequences(seq2, maxlen=max_seq_len)
padded3 = pad_sequences(seq3, maxlen=max_seq_len)
padded4 = pad_sequences(seq4, maxlen=max_seq_len)
padded5 = pad_sequences(seq5, maxlen=max_seq_len)
padded6 = pad_sequences(seq6, maxlen=max_seq_len)

# prediction 1
# start_time1 = time.time()
prediction1 = loaded_model_v2.predict(padded1)  
# prediction 2
# start_time2 = time.time()
prediction2 = loaded_model_v2.predict(padded2)
# prediction 3
# start_time3 = time.time()
prediction3 = loaded_model_v2.predict(padded3)
# prediction 4
# start_time4 = time.time()
prediction4 = loaded_model_v2.predict(padded4)
# prediction 5
# start_time5 = time.time()
prediction5 = loaded_model_v2.predict(padded5)
# prediction 6
# start_time6 = time.time()
prediction6 = loaded_model_v2.predict(padded6)

print('Message-1: ' + str(message1))
print('prediction-1: {}'.format(class_names[np.argmax(prediction1)]))
print('')

print('Message-2: ' + str(message2))
print('prediction-2: {}'.format(class_names[np.argmax(prediction2)]))
print('')

print('Message-3: ' + str(message3))
print('prediction-3: {}'.format(class_names[np.argmax(prediction3)]))
print('')

print('Message-4: ' + str(message4))
print('prediction-4: {}'.format(class_names[np.argmax(prediction4)]))
print('')

print('Message-5: ' + str(message5))
print('prediction-5: {}'.format(class_names[np.argmax(prediction5)]))
print('')

print('Message-6: ' + str(message6))
print('prediction-6: {}'.format(class_names[np.argmax(prediction6)]))

Message-1: ['I was very happy when my scholarship to go to flight school was approved after it had been cancelled']
prediction-1: joy

Message-2: ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled, but now I'm sad that my dad died"]
prediction-2: sadness

Message-3: ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled, but now I'm sad that my dad died and now I just won the lottery so I will celebrate my victory and I am very thrilled about it"]
prediction-3: joy

Message-4: ["I was very happy when my scholarship to go to flight school was approved after it had been cancelled, but now I'm sad that my dad died and now I just won the lottery so I will celebrate my victory and I am very thrilled about it but then found out It all goes to my ex wife. I think I'm depressed now"]
prediction-4: sadness

Message-5: ["I was very happy when my scholarship to go to flight school was approved afte